In [ ]:
#!pip install pyglet==1.5.27


In [2]:
import gym

from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env
import numpy as np
from game.simulator import PricingGame
from agent.random_agent import RandomAgent

#from pyglet.gl import *

# # Parallel environments
# env = make_vec_env("CartPole-v1", n_envs=4)

# model = A2C("MlpPolicy", env, verbose=1)
# model.learn(total_timesteps=25000)
# model.save("a2c_cartpole")

# del model # remove to demonstrate saving and loading

# model = A2C.load("a2c_cartpole")

# obs = env.reset()
# while True:
#     action, _states = model.predict(obs)
#     obs, rewards, dones, info = env.step(action)
#     #env.render()

In [ ]:
from game.pricing_env import PricingGameEnv
from data.generator import SimpleDemandGenerator
from functools import partial
np.seterr(all='raise') 


n = 100
n_envs = 10
generator = SimpleDemandGenerator()
env_class = partial(PricingGameEnv, generator, n)

env = make_vec_env(env_class, n_envs=n_envs)



model = A2C("MultiInputPolicy", env, verbose=1)
model.learn(total_timesteps=5000)

obs = env.reset()
done = [False]
rewards = []
states = []
actions = []
cws = []
total_reward = [0.0]*n_envs
while not all(done):
    action, _states = model.predict(obs)
    #print(action)
    obs, rewards, done, info = env.step(action)
    #env.render(mode="plotly")
    states.append(env.envs[0].env.get_state())
    print("cw", env.envs[0].env.current_cw)
    print("rew", rewards[0])
#     print("acc revenues",sum([r.sum() for r in env.envs[0].env.revenues]) )
    actions.append(action[0])
    total_reward+=rewards
print(total_reward)

Using cpu device


In [ ]:
states[-2].visualize()

In [ ]:
env2 = PricingGameEnv(generator, n)
simulator = PricingGame(generator, n)
agent =  RandomAgent()


In [ ]:
state = None
reward = 0
done = False
env2.reset()
total_reward = 0
while not done:
    discounts =  np.float32([10.0]*100)#agent.act(simulator)
    obs, reward, done, info = env2.step(np.array(discounts))
    total_reward+=reward
print(total_reward)
env2.get_state().visualize()

In [ ]:
print(env2.get_final_score())


In [ ]:
[r.sum() for r in env2.sales]

In [ ]:
 (env2.stocks[-1] * env2.black_prices).sum()